# This nb is used to download data from influxdb into csv files

## Imports

In [ ]:
import time
import pandas as pd
from InfluxDBInterface.libdb import download_data_DB

## Configuration

In [2]:
first_dt = "2025-10-01T00:00:00Z"
last_dt = "2025-10-06T10:30:00Z"
res = 60 # resolution in seconds.
# Here we don't need high resolution as we are dealing with pictures too (sampled every 5min)
# But we need it high enough because we use "last" to aggregate data.
bucket = "a_sensing"
filters = {
    "measurement": ["co2", "htr", "tmp"],
    "hive_num": 1
}
filename = f"{time.strftime('%Y-%m-%d_%H-%M')}_influxdb_data"
destination = "data"
verbose = False

## Querying

In [3]:
start_ts = pd.Timestamp(first_dt)
end_ts = pd.Timestamp(last_dt)
print(f"Downloading data from {first_dt} to {last_dt} with a resolution of {res} seconds")
data = download_data_DB(bucket, start_ts, end_ts, res, filters, destination, filename, verbose)